# The First Exponential Quantum Advantage: Deutsch Jozsa Algorithm

The Deutsch-Jozsa algorithm proposed in <a href="doi:10.1098/rspa.1992.0167">[1]</a> helps us identify whether a boolean function is balanced or constant. 

* Boolean Function: A mathematical function whose inputs and outputs assume values from a binary or two-element set of the fashion {True, False} or {0,1} and is of the form $f: \{0,1\}^k \to \{0,1\}$ where k is any non-negative integer.

* Balanced vs Constant functions: A balanced function produces outputs such that there are equal number of outcomes corresponding to each of the values in the given two-element set, whereas a constant function, as the name suggests, always produces the same output for all inputs.

Adding a dash of music to this tutorial, we shall refer to this algorithm as the DJ as there's indeed music in this algorithm that has inspired the discovery of other algorithmic 'musical compositions' like the Bernstein-Vazirani algorithm.

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/DJ1.png" alt="Deutsch Jozsa ALgorithm" />
</div>

Interestingly, DJ was the first quantum algorithm to "theoretically" demonstrate an exponential speedup over its determiniatic classical counterpart <a href="doi:10.1098/rspa.1998.0164">[2]</a>. This is because the classical solution neccessitates the need to evaluate atleast more than half the inputs to determine whether the given $f(x)$ is balanced whereas in the quantum solution, we only need to evaluate the $f(x)$ once, exploiting the concept of quantum superposition.

In the classical solution, the approach is to query the oracle multiple times. To understand the magnitude of inefficiency in such an approach, let's consider the worst case scenario. Let's say that on investigating our outputs for each of the multiple queries to the oracle, we will need to check that exactly half of all possible inputs plus one to be certain that the function is constant, which is essentially an exhaustive or brute-force search that we end up performing. This is because if the function is not constant, you will encounter a different output by the time you have tested half the inputs plus one.

On the other hand, the quantum solution boils down the solution to only one single query.The quantum solution does this by leveraging the parallelism that superposition provides to investigate the outputs for each given input to see whether the output is same irrespective of anything (constant) or whether there are equal number of two possible binary outputs. The parallelism helps us treat the problem as simply the evaluation of $f(x)$ at once.

Although there are a few known, accepted implementations, its beauty lies in the that it can be carried out in 5 steps, which we will detail in the Guided Implementation below. We start out with choosing a predicate that'll act as our function condition for our oracle. Our initial setup consists of n qubits and one ancilla qubit in the state $|0\rangle$. Two of our key components are putting our n qubits into a superposition of all possible inputs of the function using hadamard transform and an oracle constructed to represent our query, or in simple words the given function $f(x)$.

Now let's implement the DJ algorithm with Classiq! [Jump to Mathematical Description](#mathematical-description) for detailed understanding

**Classiq Concepts**
* <a href="https://docs.classiq.io/latest/explore/functions/qmod_library_reference/classiq_open_library/hadamard_transform/hadamard_transform/">Hadamard Transform</a>
* <a href="https://docs.classiq.io/latest/reference-manual/platform/qmod/language-reference/statements/within-apply/">Within Apply</a>

<details markdown>
<summary markdown>Information for New Classiq Users</summary>
 If you're new to Classiq, check out our <a href="https://docs.classiq.io/latest/classiq_101/">Classiq 101</a> guide to help you get started and join our <a href="https://classiq-community.slack.com/join/shared_invite/zt-2kbrc5yjn-rXv1pg9F_QhFH~aqUMFm0Q#/shared-invite/email">Slack Community</a> to get first-hand support from our team of experts to grow in your quantum journey with Classiq!

#### Guided Implementation

**Step 1:**  The first step in the algorithm is to encode the function that should be evaluated if it is balanced or constant into the quantum algorithm. As a specific example, a balanced function is demonstrated here. The function is evaluated to be 1 if the quantum number x is greater than the median value of all possible inputs, otherwise it is evaluated to be 0. Upon evaluation, the qubit variable res is initailized with the function output value.

Let's consider the predicate, res ^= x > 7, for our algorithm. 
This means that 
* whenever x > 7 is True, it evaluates 1 and res is XORed with 1, effectively toggling it to 0 or 1. 
* whenever x > 7 is False, res is XORed with 0, effectively always leaving its value unchanged

In simple words, this predicate is the question we want our oracle to answer. As you can clearly see in this example, the first scenario corresponds to the function being balanced and the latter to being constant. 

In [3]:
from classiq import *
@qfunc
def simple_predicate(x: QNum, res: QBit):
  res ^= x > 7

**Step 2:** Initial state preparation

Now, we need two registers in our initial setup
* First register: Minus state $|-\rangle$ (Done using [Step 3](#step3))
* Second register: A superposition of all possible states (Done in the first hadamard transformation in [Step 3](#step4))
Let's prepare the initial minus state $|-\rangle$ by applying the Pauli X and Hadamard gates. 

<details markdown>
<summary markdown> HINT </summary>


Use `H(x)`,`X(x)`. While H(x) applies a hadamard gate, helping put quubits in superposition, X(x) corresponds to the Pauli X gate that flips the given qubit

In [4]:
@qfunc
def prepare_minus(out: Output[QBit]):
    allocate(1, out)
    X(out)
    H(out)

<a id="step3"></a>**Step 3:** We're now ready to prepare the DJ's oracle which inturn changes the phase of each basis state depending on the output of the function for that input. 
The function takes two named inputs: 1. compute -> A `lambda` function that holds the preparation and uncompute function. In this case the `prepare_minus` function. 2. action -> A `lambda` function that holds the action that is wrapped within the compute functions which corresponds to the `simple_predicate` function . In this case that is the entire `apply_oracle`.

In [5]:
@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: simple_predicate(x, aux)
    )

<details markdown>
<summary markdown> NOTE </summary>
Here's a <a href="https://docs.classiq.io/latest/reference-manual/platform/qmod/language-reference/operators/#lambda-functions">short tutorial</a> on the usage of lambda in python function if you're unfamiliar.

<a id="step4"></a>**Step 4 and 5:** Let's bring it all together and as a last step, measure the system. We want to put the qubits in superposition of all possible inputs in their respective computational basis before and after the application of the oracle. This is done by hadamard transformation. It's nice to think of it like you're making a delicious quantum sandwich with 2 hadamard transformations!

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/D2.png" alt="Deutsch Jozsa Algorithm" />
</div>


In [6]:
@qfunc
def main(x: Output[QNum]):
    allocate(4, x)
    within_apply(compute=lambda: hadamard_transform(x), action=lambda: apply_oracle(x))

<details markdown>
<summary markdown> NOTE </summary>
It's interesting to observe here that since the algorithm involves applying two hadamard transformations, which are unitary in nature, as a sandwich of the oracle (Read Mathematical Description below to learn more), we can exploit the convenience of our in-built within_apply() function with our oracle as the action and the hadamard transform as the compute

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/DJ_execution.mp4" alt="Deutsch Jozsa Algorithm" />
</div>

You can see here that the while the application of haramard transformations before and after the oracle. But what does the oracle itself do? We can expand to see its inner workings - As we used a within_apply in the oracle for the creation of minus state, we can see a <code>minus_state_0</code> and a <code>minus_state_1</code> sandwiching our constant function, which acts as our predicate.

Time to synthesize our main function into a quantum program

In [7]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/3995daa8-9c87-4457-a987-bf4a9e37e3c6?version=0.42.2


<a id="mathematical-description"></a>
#### Mathematical Description

**Initialization**

First, $n$ qubits in the state $|0\rangle^{\otimes n}$ and one ancilla qubit in the state $|0\rangle$ are prepared. The initial state of the system would look like $|0\rangle^{\otimes n}$ $|1\rangle$. To the ancilla qubit, a Pauli $X$ gate (to flip the ancilla $|0\rangle$ to $|1\rangle$) followed by a Hadamard gate $H$, effectively transforming it into the $|-\rangle$ state. After this transformation, the initial state of the system would be $|0\rangle^{\otimes n} \otimes |-\rangle$



**Hadamard Transformation** (Bread)

This transformation endeavors putting the $n$ qubits in the state $|0\rangle$ into a superposition of all possible input states by applying a $H$ gate to each qubit. This puts each qubit in a superposition of the form $\frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)$ and the entire system would be $\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n - 1} |x\rangle$ where |$x\rangle$ shows the computational basis we're dealing with or, in simple words, it's all possible states of the qubits in the given basis.

**Querying the Oracle** (Sandwich Filling)

Remember that we started the tutorial by assuming our function or predicate to be $res \oplus  x > 7$. For the oracle, the function would be perform the $XOR$ operation based on its predicate condition $x > 7$.Thus the oracle can be written as $U_f |x\rangle = |x\rangle \otimes |f(x)\rangle$, which means that the oracle corresponds to  $f(x) = \text{res} \oplus (x > 7)$. Thus, applying the oracle $U_f$ takes $\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n - 1} |x\rangle$ to $\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n - 1} (-1)^{f(x)} |x\rangle$.

Note that $f(x) = 1$ if it's balanced and $f(x) = 0$ if it's constant.


**Another Hadamard Transform** (Bread)

On the above obtained state, we apply another hadamard transformation. Here's how you can do this operation

$H^{\otimes n} \left( \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n - 1} (-1)^{f(x)} |x\rangle \right)$ which results in $\frac{1}{2^{n/2}} \sum_{z=0}^{2^n - 1} \left( \sum_{x=0}^{2^n - 1} (-1)^{f(x) + x \cdot z} \right) |z\rangle$, where $z$ denotes all possible inputs of this new superposition in the given computational basis, which will also be our output bitstring upon measurement.

It's noteworthy to observe that this is the negation of a constructive and destructive interference.

**Lastly, Let's Measure!**

Measurement would result in a $n$ bit z bitstring. It can now be trivial to recognize whether the function is constant or balanced.

If z is all zeroes, $z = 000...0$, we can conclude that the function $f(x)$ is constant.
Else, if $z \neq 000...0$, we can conclude that the function $f(x)$ is balanced.

#### All Code Together

In [8]:
from classiq import *

@qfunc
def constant_function(x: QNum, res: QBit):
    res ^= x < (2**x.size)


@qfunc
def prepare_minus(out: Output[QBit]):
    allocate(1, out)
    X(out)
    H(out) 


@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: constant_function(x, aux)
    )

@qfunc
def main(x: Output[QNum]):
    allocate(4, x)
    within_apply(compute=lambda: hadamard_transform(x), action=lambda: apply_oracle(x))

quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)


Opening: https://platform.classiq.io/circuit/d3a337bf-1fe1-4dcb-b960-f19a15c8ff28?version=0.42.2


#### Results

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/DJ_results.png" alt="Deutsch Jozsa Algorithm" />
</div>

In the results, we can see that the binary string above denotes our result 1000 and below are the number of shots we ran the quantum program for and the value of x = 8. It's clear that the result 1000 conveys that the function is not constant but balanced. Let's investigate into why the result means our function $f(x)$ is balanced:

* The second Hadamard transform did not return all $|0\rangle$ states, which would be the case for a constant function.

* The non-zero result (1000) indicates the presence of phase differences due to the balanced nature of the function, resulting in an interference pattern where at least one qubit is $∣1\rangle$.

#### Congratulations!
You have successfully implemented your first Deutsch Jozsa Algorithm. Feel free to explore with your own predicates to master DJ algorithm further.

#### References

\[1\] David Deutsch and Richard Jozsa (1992). "Rapid solutions of problems by quantum computation". Proceedings of the Royal Society of London A. 439: 553–558. <a href="doi:10.1098/rspa.1992.0167">doi:10.1098/rspa.1992.0167</a>

\[2\] R. Cleve; A. Ekert; C. Macchiavello; M. Mosca (1998). "Quantum algorithms revisited". Proceedings of the Royal Society of London A. 454: 339–354. <a href="doi:10.1098/rspa.1998.0164">doi:10.1098/rspa.1998.0164</a>